In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file,'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('training set',train_dataset.shape,train_labels.shape)
    print('validation set',valid_dataset.shape,valid_labels.shape)
    print('test set',test_dataset.shape,test_labels.shape)

training set (200000, 28, 28) (200000,)
validation set (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset,labels):
    dataset = dataset.reshape((-1,image_size*image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset,labels
train_dataset,train_labels = reformat(train_dataset,train_labels)
valid_dataset,valid_labels = reformat(valid_dataset,valid_labels)
test_dataset,test_labels = reformat(test_dataset,test_labels)
print('training set',train_dataset.shape,train_labels.shape)
print('validation set',valid_dataset.shape,valid_labels.shape)
print('test set',test_dataset.shape,test_labels.shape)

training set (200000, 784) (200000, 10)
validation set (10000, 784) (10000, 10)
test set (10000, 784) (10000, 10)


In [4]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset[:train_subset,:])
    tf_test_dataset = tf.constant(test_dataset[:train_subset])
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size , num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset,weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(tf.matmul(tf_valid_dataset , weights) + biases)
    test_predictions = tf.nn.softmax(tf.matmul(tf_test_dataset , weights) + biases)

In [5]:
num_steps = 500

def accuracy(predictions , labels):
    return(100.0 * np.sum(np.argmax(predictions,1) == np.argmax(labels,1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('initialized')
    for step in range(num_steps):
        _,l,predictions = session.run([optimizer,loss,train_predictions])
        if (step % 100 == 0):
            print('loss at step %d %f' % (step,l))
            print('training accuracy: %.1f%%' % accuracy(predictions,train_labels[:train_subset,:]))
            print('validation accuracy : %.1f%%' % accuracy(valid_predictions.eval(),valid_labels))
    print('test accuracy : %.1f%%' % accuracy(test_predictions.eval(),test_labels))

initialized
loss at step 0 20.044138
training accuracy: 7.0%
validation accuracy : 9.6%
loss at step 100 2.255458
training accuracy: 71.4%
validation accuracy : 71.3%
loss at step 200 1.841102
training accuracy: 74.4%
validation accuracy : 73.3%
loss at step 300 1.615429
training accuracy: 75.8%
validation accuracy : 74.2%
loss at step 400 1.460021
training accuracy: 76.6%
validation accuracy : 74.5%
test accuracy : 81.9%


In [24]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset,weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits) 
                          + 0.001 * tf.nn.l2_loss(weights) 
                          + 0.001 * tf.nn.l2_loss(biases)) 
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(tf.matmul(valid_dataset,weights) + biases)
    test_predictions = tf.nn.softmax(tf.matmul(test_dataset,weights) + biases)
    

In [26]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
   
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_predictions], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_predictions.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_predictions.eval(), test_labels))


Initialized
Minibatch loss at step 0: 21.337368
Minibatch accuracy: 3.1%
Validation accuracy: 10.7%
Minibatch loss at step 500: 2.910050
Minibatch accuracy: 73.4%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.577163
Minibatch accuracy: 82.8%
Validation accuracy: 77.9%
Minibatch loss at step 1500: 1.006680
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 2000: 0.786786
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Test accuracy: 87.8%
